In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from bayesian_model import metropolis_hastings
from knn_baseline import iris 

In [4]:
data = iris.data
target = iris.target

In [6]:
metropolis_hastings(target, data, niter=100, method="pseudo_likelihood")

(3.5638128541036083, 20)

In [7]:
metropolis_hastings(target, data, niter=100, method="pseudo_likelihood")

100%|██████████| 100/100 [01:10<00:00,  1.41it/s]


(3.328036391176332, 9)

In [8]:
metropolis_hastings(target, data, niter=int(1e4), method="pseudo_likelihood")

TypeError: 'float' object cannot be interpreted as an integer